In [1]:
from simulation import citygraph_dataset
# from learning import inductive_route_learning, eval_route_generator, bee_colony
from learning.bee_colony import main as main_bee  # я так обозвал
from learning.eval_route_generator import main as main_eval # я так обозвал
from omegaconf import OmegaConf, DictConfig
from simulation import drawing

from tqdm import tqdm
from pathlib import Path

In [ ]:
dataset = citygraph_dataset.DynamicCityGraphDataset(
    min_nodes=25,
    max_nodes=25,
    edge_keep_prob=0.7,
    data_type=citygraph_dataset.MIXED,  # or any other type you want
    directed=False,
    fully_connected_demand=True,  # default SIDE_LENGTH_M
    mumford_style=True,
    pos_only=False
)

# Generate graphs
n_graphs = 100  # number of graphs you want to generate
graphs = [dataset.generate_graph(draw=False) for _ in tqdm(range(n_graphs))]

100%|██████████| 100/100 [00:00<00:00, 209.67it/s]


In [4]:
import pickle
from pathlib import Path

# Путь для сохранения
save_path = Path('./output_graphs')
if not save_path.exists():
    save_path.mkdir(parents=True)

# Сохраняем объект в файл
with open(save_path / 'raw_graphs_1000.pkl', 'wb') as ff:
    pickle.dump(graphs, ff)


In [5]:
import pickle

with open(save_path / 'raw_graphs_1000.pkl', 'rb') as f:
    graphs = pickle.load(f)

In [6]:
from hydra import initialize_config_dir, compose
from omegaconf import OmegaConf
import os

cfg_dir = os.path.abspath("../TNDP_learning/cfg")

### Обучение LC

In [25]:
with initialize_config_dir(config_dir=cfg_dir, job_name="app"):
    cfg_learn = compose(config_name="ppo_20nodes_copy.yaml", # в этот файл можно положить путь к пикл файлу с графами, также можно попробовать другие параметры на обучение
                            overrides=["+run_name=random_graphs"]) # при желании можно накинуть ему имя 


/tmp/ipykernel_38100/2784490261.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=cfg_dir, job_name="app"):


In [26]:
print(OmegaConf.to_yaml(cfg_learn)) # чисто проверка как выглядит конфиг 

ppo:
  n_iterations: 200
  val_period: 10
  n_epochs: 1
  minibatch_size: 16
  horizon: 120
  epsilon: 0.2
  use_gae: true
  gae_lambda: 0.95
discount_rate: 0.95
diff_reward: true
baseline_lr: 0.0005
entropy_weight: 0.0
batch_size: 16
reward_scale: 1.0
lr: 0.0016134816080499328
decay: 0.0008404361781997002
optimizer: Adam
eval:
  n_routes: 7
  min_route_len: 2
  max_route_len: 8
dataset:
  type: pickle
  kwargs:
    path: datasets/20_nodes/mixed
    space_scale: 0.6
    demand_scale: 0.2
experiment:
  logdir: training_logs
  anomaly: false
  cpu: false
  seed: 0
  symmetric_routes: true
  cost_function:
    type: mine
    kwargs:
      mean_stop_time_s: 0
      avg_transfer_wait_time_s: 300
      demand_time_weight: 0.33
      route_time_weight: 0.33
      median_connectivity_weight: 0.33
      constraint_violation_weight: 5.0
      variable_weights: true
      pp_fraction: 0.15
      op_fraction: 0.15
      mcw_fraction: 0.15
model:
  common:
    dropout: 0.0
    nonlin_type: ReLU
   

In [27]:
from learning import inductive_route_learning
inductive_route_learning.setup_and_train(cfg_learn) # веса для модели вернутся в папку output, там будет файл .pt / при желании можно накинуть ему имя 



  0%|          | 0/200 [03:01<?, ?it/s]






KeyboardInterrupt: 

### Стартовый набор маршрутов

In [ ]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    
    cfg_eval = compose(
        config_name="eval_model_mumford",   # из @hydra.main
        overrides=[
            "+eval=mandl", # конфиг в котором задается кол-во маршрутов и их макс и мин длины (можно заменить на vo для теста по ваське)
            "+model.weights=../TNDP_learning/output/inductive_train_1_checkpoints/iter199.pt", # путь к весам модели
            "++run_name=test" # имя запуска, вернет pickle файл с тензором в output_routes
        ]
    )

In [ ]:
print(OmegaConf.to_yaml(cfg_eval)) # чисто проверка как выглядит конфиг 

experiment:
  logdir: null
  anomaly: false
  cpu: false
  seed: 0
  symmetric_routes: true
  cost_function:
    type: mine
    kwargs:
      mean_stop_time_s: 0
      avg_transfer_wait_time_s: 300
      demand_time_weight: 0.33
      route_time_weight: 0.33
      median_connectivity_weight: 0.33
      constraint_violation_weight: 5.0
      variable_weights: true
      pp_fraction: 0.33
      op_fraction: 0.33
model:
  common:
    dropout: 0.0
    nonlin_type: ReLU
    embed_dim: 64
  route_generator:
    kwargs:
      force_linking_unlinked: false
      logit_clip: null
      n_nodepair_layers: 3
      n_pathscorer_layers: 3
      pathscorer_hidden_dim: 16
      n_halt_layers: 3
      halt_scorer_type: endpoints
      serial_halting: true
    type: PathCombiningRouteGenerator
  backbone_gn:
    net_type: graph attn
    kwargs:
      n_layers: 5
      in_node_dim: 4
      in_edge_dim: 14
      use_norm: false
      n_heads: 4
      dense: false
  weights: ../TNDP_learning/output/induct

In [ ]:
main_eval(cfg_eval)

100%|██████████| 1/1 [00:42<00:00, 42.67s/it]

,10.000,2.046,16.303,515.160,25.027,46.836,14.273,13.863,724.500,0.000,0.387,42.616,0.000



/home/ruslan/TNDP_learning/learning/utils.py:320: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  out_stats = (final_costs.mean(), final_costs.std(), metrics)


In [ ]:
# Загрузка объекта из файла
import pickle
with open('/home/ruslan/TNDP_learning/output_routes/nn_construction_test_routes.pkl', 'rb') as f:
    routes = pickle.load(f)[0].tolist()

In [ ]:
for e in routes:
    print(e)

[4, 1, 9, 11, 14, 12, 13, 15, 33, 32, 55, 56, 59, 84, 83, 104, 103, 106, 107, 105, 101, 97, 92, 93, 95]
[21, 40, 37, 3, 63, 61, 62, 85, 83, 104, 99, 98, 94, 78, 75, 74, 73, 72, 70, 68, 66, 45, 43, 41, -1]
[0, 1, 9, 11, 14, 12, 10, 7, 30, 29, 28, 53, 54, 80, 77, 75, 87, 88, -1, -1, -1, -1, -1, -1, -1]
[57, 60, 3, 36, 35, 34, 31, 2, 29, 27, 25, 24, 23, 22, 42, 41, 43, 45, 66, 68, 70, 72, 76, 54, -1]
[39, 18, 20, 19, 17, 16, 31, 2, 29, 28, 26, 51, 49, 66, 64, 65, 67, 71, -1, -1, -1, -1, -1, -1, -1]
[82, 79, 77, 76, 52, 51, 50, 46, 44, 47, 48, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[95, 93, 92, 97, 101, 105, 107, 106, 103, 104, 83, 84, 59, 56, 55, 32, 33, 15, 13, 12, 14, 11, 9, 1, 0]
[11, 14, 12, 10, 7, 30, 29, 28, 53, 54, 76, 73, 69, 86, 87, 88, 91, 96, 100, 102, 101, 97, 92, 90, -1]
[95, 93, 92, 97, 101, 105, 107, 106, 103, 104, 83, 84, 59, 56, 55, 32, 33, 15, 13, 12, 14, 11, 9, 1, 4]
[4, 1, 9, 11, 14, 12, 13, 15, 33, 32, 55, 56, 59, 84, 83, 104, 103, 106, 107, 105, 101,

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# 1. Загрузим координаты
coord_file = "CEC2013Supp/Instances/VOCoords.txt"

# 1. Загрузка координат
with open(coord_file, "r") as f:
    lines = f.readlines()[1:]  # Пропускаем первую строку (кол-во точек)
coords = [list(map(float, line.strip().split())) for line in lines]
df = pd.DataFrame(coords, columns=["x", "y"])

# 2. Создание GeoDataFrame в системе UTM (метры)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["x"], df["y"]),
    crs="EPSG:32636"  # UTM зона 35N, Санкт-Петербург
)

# # 3. Преобразование в WGS84 (широта/долгота)
gdf = gdf.to_crs("EPSG:4326")
gdf['id'] = gdf.index

In [ ]:
import geopandas as gpd
import folium
from shapely.geometry import LineString
import random
import math

def plot_routes_on_map(routes, gdf):
    # Получаем координаты точек
    coords = {row['id']: (row.geometry.y, row.geometry.x) for _, row in gdf.iterrows()}

    # Центр карты
    center_lat = gdf.geometry.y.mean()
    center_lon = gdf.geometry.x.mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Добавляем вершины
    for _, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=f"ID: {row['id']}"
        ).add_to(m)

    # Функция генерации случайного цвета
    def random_color():
        return "#{:06x}".format(random.randint(0, 0xFFFFFF))

    # Функция для смещения маршрута
    def offset_route(coords, offset_meters):
        if len(coords) < 2:
            return coords

        offset_coords = []
        for i in range(len(coords)):
            lat, lon = coords[i]

            if i == 0:
                lat2, lon2 = coords[i + 1]
            else:
                lat2, lon2 = coords[i - 1]

            dx = lon2 - lon
            dy = lat2 - lat

            length = math.sqrt(dx**2 + dy**2)
            if length == 0:
                nx, ny = 0, 0
            else:
                nx = -dy / length
                ny = dx / length

            delta_lat = (offset_meters * ny) / 111111
            delta_lon = (offset_meters * nx) / (111111 * math.cos(math.radians(lat)))

            offset_coords.append((lat + delta_lat, lon + delta_lon))

        return offset_coords

    # Добавляем маршруты с сдвигом
    for idx, route in enumerate(routes):
        route_coords = []
        for node_id in route:
            if node_id != -1 and node_id in coords:
                route_coords.append(coords[node_id])

        if len(route_coords) > 1:
            shifted = offset_route(route_coords, offset_meters=idx * 7)
            route_line = folium.PolyLine(
                locations=shifted,
                color=random_color(),
                weight=5,
                opacity=0.9,
                popup=f"Маршрут {idx}"
            )
            fg = folium.FeatureGroup(name=f"Маршрут {idx}")
            fg.add_child(route_line)
            m.add_child(fg)

    folium.LayerControl().add_to(m)
    return m


In [ ]:
map_result = plot_routes_on_map(routes, gdf)
map_result

### Генерация 

In [ ]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    cfg_bco = compose(
        config_name="bco_mumford",  # как в @hydra.main для bee_colony.py
        overrides=[
            "+eval=mandl",
            "eval.dataset.path=../TNDP_learning/datasets/mumford_dataset/Instances",
            "init.path=output_routes/nn_construction_test_routes.pkl",
            "++eval.dataset.path=../TNDP_learning/CEC2013Supp/Instances", # путь к данным (можно заменить на ваську)
            "++eval.dataset.city=VO", # название города из датасетов, при тесте на ваське написать VO
        ]
    )

In [ ]:
print(OmegaConf.to_yaml(cfg_bco)) # чисто проверка как выглядит конфиг 

n_bees: 10
n_type1_bees: 5
n_iterations: 100
batch_size: 10
force_linking_unlinked: false
experiment:
  logdir: training_logs
  anomaly: false
  cpu: false
  seed: 0
  symmetric_routes: true
  cost_function:
    type: mine
    kwargs:
      mean_stop_time_s: 0
      avg_transfer_wait_time_s: 300
      demand_time_weight: 0.33
      route_time_weight: 0.33
      median_connectivity_weight: 0.33
      constraint_violation_weight: 5.0
      variable_weights: true
      pp_fraction: 0.33
      op_fraction: 0.33
init:
  method: load
  path: output_routes/nn_construction_test_routes.pkl
eval:
  csv: true
  n_routes: 10
  min_route_len: 4
  max_route_len: 25
  dataset:
    type: mumford
    path: ../TNDP_learning/CEC2013Supp/Instances
    city: VO



In [ ]:
main_bee(cfg_bco)

100%|██████████| 1/1 [14:38<00:00, 878.66s/it]

,10.000,0.648,15.148,344.500,24.649,53.538,21.335,0.478,0.000,0.000,0.347,878.623,101.000



/home/ruslan/TNDP_learning/learning/utils.py:320: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  out_stats = (final_costs.mean(), final_costs.std(), metrics)


In [ ]:
# Загрузка объекта из файла
import pickle
with open('/home/ruslan/TNDP_learning/output_routes/bco_04_08_2025_12:51:40_routes.pkl', 'rb') as f:
    routes = pickle.load(f)[0].tolist()

In [ ]:
map_result = plot_routes_on_map(routes, gdf)
map_result

In [ ]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    cfg_neural = compose(
        config_name="neural_bco_mumford",
        overrides=[
            "+eval=mandl",
            "++eval.dataset.path=../TNDP_learning/datasets/mumford_dataset/Instances",
            "++model.weights=../TNDP_learning/output/inductive_train_1_checkpoints/iter199.pt",
            # Куда сохранять initial routes (init.method по defaults = load)
            "init.path=output_routes/nn_construction_test_routes.pkl",
            "++eval.dataset.path=../TNDP_learning/CEC2013Supp/Instances", # путь к данным (можно заменить на ваську)
            "++eval.dataset.city=VO", # название города из датасетов, при тесте на ваське написать VO
        ]
    )

In [ ]:
print(OmegaConf.to_yaml(cfg_neural)) # чисто проверка как выглядит конфиг 

n_bees: 10
n_iterations: 400
batch_size: 10
neural_bees: true
force_linking_unlinked: false
experiment:
  logdir: training_logs
  anomaly: false
  cpu: false
  seed: 0
  symmetric_routes: true
  cost_function:
    type: mine
    kwargs:
      mean_stop_time_s: 0
      avg_transfer_wait_time_s: 300
      demand_time_weight: 0.33
      route_time_weight: 0.33
      median_connectivity_weight: 0.33
      constraint_violation_weight: 5.0
      variable_weights: true
      pp_fraction: 0.33
      op_fraction: 0.33
model:
  common:
    dropout: 0.0
    nonlin_type: ReLU
    embed_dim: 64
  route_generator:
    kwargs:
      force_linking_unlinked: false
      logit_clip: null
      n_nodepair_layers: 3
      n_pathscorer_layers: 3
      pathscorer_hidden_dim: 16
      n_halt_layers: 3
      halt_scorer_type: endpoints
      serial_halting: true
    type: PathCombiningRouteGenerator
  backbone_gn:
    net_type: graph attn
    kwargs:
      n_layers: 5
      in_node_dim: 4
      in_edge_dim: 1

In [ ]:
main_bee(cfg_neural)

100%|██████████| 1/1 [2:23:50<00:00, 8630.42s/it]

,10.000,0.663,15.840,350.260,24.709,48.068,25.582,1.641,0.000,0.000,0.352,8630.392,401.000
